In [232]:
#Author: Ryan Liao
#Date: 2021/11/17
import pandas as pd
import itertools
import statsmodels.formula.api as smf
import numpy as np

In [237]:
def _pred_Contribution(df,sequence,x):
    Sq = list(sequence)
    idx = Sq.index(x) +1 
    out = df[df['Predictor Sequence']  == tuple(sequence)][f'Predictor_{idx}_Enters'] 
    if idx == 1:
        return out 
    else:
        return out -  df[df['Predictor Sequence']  == tuple(sequence)][f'Predictor_{idx-1}_Enters'] 
    

def Pred_Contributions(df,x):
    out = []
    for idx in df.index:
        row = df.iloc[idx] 
        out.append(float(_pred_Contribution(df,row['Predictor Sequence'],x)))
    return out

class SV_OLS:
    "This Class is dedicated for calculating the Shapely Value of a simple linear regression"
    def __init__(self,data,Y_name,X_names):
        self.data = data
        self.Y_name = Y_name
        self.X_names = X_names
        self.Fetch_combs()
    
    def Fetch_combs(self):
        Regressors = self.X_names
        OUT = {0:''}
        for N in range(1,len(Regressors)+1):
            OUT[N] = [i for i in itertools.combinations(Regressors,N)]
        self.combs = OUT
        
    def formulate(self,lst):
        out = self.Y_name + ' ~ 1'
        for i in lst:
            out += f' + {i}'
        return out

    def Contribution(self,candidates):
        mod = smf.ols(self.formulate(candidates),self.data)
        mod = mod.fit()
        return mod.rsquared
    
    def get_R_table(self):
        R_table = {('1'):T.Contribution('1')}
        for N in self.combs:
            for comb in self.combs[N]:
                R_table[comb] = T.Contribution(comb)
        return R_table
    
    def _get_df(self):
        #Get the permutation of grandcolision 
        Perms = [i for i in itertools.permutations(T.X_names,r=len(T.X_names))]
        #Create Framework
        _dat =  {'Predictor Sequence':[]}
        count = 0
        for x in self.X_names:
            count += 1 
            _dat[f'Predictor_{count}_Enters'] = []
        df = pd.DataFrame(data = _dat)
        #Populate values:
        for perm in Perms:
            temp_mod = set()
            row = [tuple(perm)]
            for p in perm:
                temp_mod.add(p)
                #print(R_tab[tuple(temp_mod)])
                row.append(self.Contribution(tuple(temp_mod)))
            df.loc[len(df.index)] = row
        return df
    
    def Get_SV_table(self):
        df = self._get_df()
        for x in self.X_names:
            df[x] = Pred_Contributions(df,x)
        return df
    
    def SV_Values(self):
        _df = self.Get_SV_table()
        out = dict()
        for x in self.X_names:
            out[x] = [np.average(_df[x])]
        return pd.DataFrame(out,index = ['Shapley Value'])
        
    

# Play Ground

In [234]:
# data = pd.read_csv('cars.csv')
# T = SV_OLS(data,'MSRP',['Horsepower','Weight','Length'])
data = pd.read_csv('sat.csv')
T = SV_OLS(data,'univ_GPA ',['high_GPA', 'math_SAT', 'verb_SAT', 'comp_GPA'])

In [235]:
df = T.Get_SV_table()
df

C:\Users\18066\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


,Predictor Sequence,Predictor_1_Enters,Predictor_2_Enters,Predictor_3_Enters,Predictor_4_Enters,high_GPA,math_SAT,verb_SAT,comp_GPA
0,"(high_GPA, math_SAT, verb_SAT, comp_GPA)",0.607719,0.617700,0.623580,0.889052,0.607719,0.009981,0.005880,0.265472
1,"(high_GPA, math_SAT, comp_GPA, verb_SAT)",0.607719,0.617700,0.885399,0.889052,0.607719,0.009981,0.003652,0.267699
2,"(high_GPA, verb_SAT, math_SAT, comp_GPA)",0.607719,0.622725,0.623580,0.889052,0.607719,0.000855,0.015006,0.265472
3,"(high_GPA, verb_SAT, comp_GPA, math_SAT)",0.607719,0.622725,0.887166,0.889052,0.607719,0.001885,0.015006,0.264441
4,"(high_GPA, comp_GPA, math_SAT, verb_SAT)",0.607719,0.885341,0.885399,0.889052,0.607719,0.000058,0.003652,0.277622
5,"(high_GPA, comp_GPA, verb_SAT, math_SAT)",0.607719,0.885341,0.887166,0.889052,0.607719,0.001885,0.001825,0.277622
6,"(math_SAT, high_GPA, verb_SAT, comp_GPA)",0.439282,0.617700,0.623580,0.889052,0.178418,0.439282,0.005880,0.265472
7,"(math_SAT, high_GPA, comp_GPA, verb_SAT)",0.439282,0.617700,0.885399,0.889052,0.178418,0.439282,0.003652,0.267699
8,"(math_SAT, verb_SAT, high_GPA, comp_GPA)",0.439282,0.470222,0.623580,0.889052,0.153358,0.439282,0.030940,0.265472
9,"(math_SAT, verb_SAT, comp_GPA, high_GPA)",0.439282,0.470222,0.887187,0.889052,0.001865,0.439282,0.030940,0.416965


In [236]:
SVs = T.SV_Values()
SVs

C:\Users\18066\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


{'high_GPA': [0.19733273502447066], 'math_SAT': [0.1152790912585573], 'verb_SAT': [0.1118682687845864], 'comp_GPA': [0.464571482342169]}


,high_GPA,math_SAT,verb_SAT,comp_GPA
Shapley Value,0.197333,0.115279,0.111868,0.464571


# Sanity Check

In [238]:
sum(SVs.loc['Shapley Value'])

0.8890515774097834

In [239]:
T.get_R_table()

{'1': 0.0,
 ('high_GPA',): 0.6077186589199637,
 ('math_SAT',): 0.4392822243998741,
 ('verb_SAT',): 0.42289166408610523,
 ('comp_GPA',): 0.8818071644490746,
 ('high_GPA', 'math_SAT'): 0.6177000327613267,
 ('high_GPA', 'verb_SAT'): 0.6227248011618057,
 ('high_GPA', 'comp_GPA'): 0.8853411406246723,
 ('math_SAT', 'verb_SAT'): 0.47022172893021674,
 ('math_SAT', 'comp_GPA'): 0.8823543653523764,
 ('verb_SAT', 'comp_GPA'): 0.8861124015627599,
 ('high_GPA', 'math_SAT', 'verb_SAT'): 0.6235798325706661,
 ('high_GPA', 'math_SAT', 'comp_GPA'): 0.8853994849379315,
 ('high_GPA', 'verb_SAT', 'comp_GPA'): 0.8871661755243562,
 ('math_SAT', 'verb_SAT', 'comp_GPA'): 0.8871866024986483,
 ('high_GPA', 'math_SAT', 'verb_SAT', 'comp_GPA'): 0.8890515774097834}